In [11]:
from dataset import get_area, get_border
from pinn import PINNModel
from models import model1
from accuracy import AccuracyCalc
import tensorflow as tf

In [12]:
# force gpu use
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

In [13]:
grid_size = (50, 50)
plotting_grid_size = (200, 200)
x_limits = (0, 2)
y_limits = (0, 2)
border_grid_size = (50, 50)

EPOCHS = 4000
EPRINT = 500
LOSS = 1e-2  # 1e-4

In [14]:
area = get_area(grid_size, x_limits, y_limits)
border = get_border(border_grid_size, x_limits, y_limits)

model = PINNModel(model1(), optm = tf.keras.optimizers.Adam(learning_rate=0.001))
model.train(1, area, border, EPOCHS, LOSS, EPRINT)

epoch: 0 loss: 101.807098
epoch: 500 loss: 0.247888148
epoch: 1000 loss: 0.0487272777
epoch: 1500 loss: 0.0179323293
epoch: 2000 loss: 0.0119290464
epoch: 2500 loss: 0.00641713


In [15]:
import numpy as np


def real_u(area):
    area = np.array(area)
    if len(area.shape) >= 3:
        x = area[0]
        y = area[1]
        return np.sin(np.pi * x) * np.sin(np.pi * y)
    else:
        return np.array([np.sin(np.pi * x) * np.sin(np.pi * y) for x, y in area])

# pred_coord = list()
# for _x in x[0]:
#     for _y in x[0]:
#         pred_coord.append([_x, _y])
# pred_u = model.predict(np.array(pred_coord)).ravel().reshape(plotting_grid_size[0], plotting_grid_size[1])
calc = AccuracyCalc(model._model, real_u, area=area)
print("Error:", calc.mse())
print("Error:", calc.maxe())

72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 638us/step 
Error: 7.764508170050296e-05
Error: 0.04048428314082545
